<a href="https://colab.research.google.com/github/nwsugz/Differentially-Private-Deep-Learning/blob/main/emotion_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import torch
import torch.nn as nn
from torchvision import transforms
from vit_pytorch import ViT
from torchvision.models import densenet121
import google.generativeai as genai
import numpy as np
import threading

# Google API key 설정
GOOGLE_API_KEY = "your_google_api_key"
genai.configure(api_key=GOOGLE_API_KEY)
model_api = genai.GenerativeModel('gemini-pro')
chat = model_api.start_chat(history=[])

# 하이퍼파라미터 설정
num_classes = 4  # 감정 클래스 수

# 데이터 전처리 설정
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 모델 정의: DenseNet-121 + ViT
class HybridModel(nn.Module):
    def __init__(self, num_classes):
        super(HybridModel, self).__init__()
        self.densenet = densenet121(pretrained=False)
        num_ftrs = self.densenet.classifier.in_features
        self.densenet.classifier = nn.Identity()  # Fully connected layer 제거

        self.vit = ViT(
            image_size=224,
            patch_size=32,
            num_classes=1024,  # 중간 차원으로 1024 사용
            dim=1024,
            depth=6,
            heads=16,
            mlp_dim=2048,
            dropout=0.1,
            emb_dropout=0.1
        )

        self.fc = nn.Linear(num_ftrs + 1024, num_classes)  # 결합된 특성에 맞게 조정

    def forward(self, x):
        densenet_features = self.densenet(x)  # Shape: (batch_size, num_ftrs)
        vit_features = self.vit(x)  # Shape: (batch_size, 1024)
        combined_features = torch.cat((densenet_features, vit_features), dim=1)  # Shape: (batch_size, num_ftrs + 1024)
        out = self.fc(combined_features)
        return out

# 모델 초기화 및 가중치 로드
device = torch.device("cpu")
model = HybridModel(num_classes=num_classes).to(device)
model.load_state_dict(torch.load('best_hybrid_model_densenet.pth', map_location=device))
model.eval()

# OpenCV 웹캠 초기화
cap = cv2.VideoCapture(0)

# 감정 레이블
emotion_labels = ['anger', 'happy', 'panic', 'sadness']

# 감정 및 프레임 감지 함수
def detect_emotion(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    emotions = []
    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
        face_img = transform(face_img).unsqueeze(0).to(device)
        outputs = model(face_img)
        _, predicted = torch.max(outputs, 1)
        emotion = emotion_labels[predicted.item()]
        emotions.append(emotion)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, emotion, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    return emotions, frame

# 얼굴 감지기 로드
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 채팅 입력을 처리하는 스레드 함수
def handle_chat_input():
    while True:
        chat_input = input("Enter your chat: ")
        if emotions:
            emotion_message = f"My emotion today is {', '.join(emotions)}. {chat_input}"
            response = chat.send_message(emotion_message)
            print(response.text)

# 채팅 입력 스레드 시작
chat_thread = threading.Thread(target=handle_chat_input, daemon=True)
chat_thread.start()

# 웹캠에서 프레임 읽기
while True:
    ret, frame = cap.read()
    if not ret:
        break

    emotions, frame = detect_emotion(frame)

    # 프레임 표시
    cv2.imshow('Emotion Detection', frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 종료
cap.release()
cv2.destroyAllWindows()


C:\Users\fud10\anaconda3\envs\khb\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\fud10\anaconda3\envs\khb\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Enter your chat:  나 오늘 뭐할까?


오늘 슬픔을 느낀다면 다음과 같은 활동을 통해 기분을 나아질 수 있습니다.

* **사랑하는 사람에게 연락하세요.** 친구, 가족, 파트너와 이야기하면 기분이 나아지고 연결감을 느낄 수 있습니다.

* **자연 속에서 시간 보내기.** 산책이나 하이킹을 나가거나 공원이나 해변을 방문하세요. 자연의 아름다움은 마음을 진정시키고 명확하게 생각하는 데 도움이 됩니다.

* **취미 즐기기.** 취미는 우리에게 기쁨과 성취감을 줄 수 있습니다. 그림, 글쓰기, 음악 연주 등 오늘 좋아하는 것을 하세요.

* **감사일기 쓰기.** 감사일기에 감사하는 것 세 가지를 매일 쓰면 긍정성과 행복감이 증가하는 것으로 나타났습니다.

* **좋아하는 음악 듣기.** 음악은 감정을 표현하고 기분을 바꾸는 강력한 도구입니다. 좋아하는 음악을 듣고 편안해지세요.

* **운동하기.** 운동은 엔도르핀을 방출하여 기분을 좋게 합니다. 하루 중 시간을 할애하여 운동을 해 보세요.

* **의미 있는 일하기.** 자원봉사를 하거나 가족이나 친구를 도와주는 등 의미 있는 일을 하면 목적 의식과 성취감이 생깁니다.

* **전문가의 도움 구하기.** 슬픔이 지속되거나 압도적이라면 상담사나 치료사에게 도움을 구하세요. 전문가는 슬픔을 처리하고 대처 메커니즘을 개발하는 데 도움을 줄 수 있습니다.
